# Personalize Text to Image Models with DreamBooth

In [ ]:
# install
!pip install -q diffusers transformers accelerate safetensors bitsandbytes datasets ftfy gradio huggingface_hub
!pip install -q git+https://github.com/huggingface/diffusers.git

In [ ]:
# imports
import os
from PIL import Image
import torch
import matplotlib.pyplot as plt

from diffusers import StableDiffusionPipeline
from diffusers.pipelines.stable_diffusion import StableDiffusionTrainingPipeline
from diffusers import DDPMScheduler
from diffusers import StableDiffusionPipeline
from diffusers import DreamBoothPipeline
from diffusers.training_utils import EMAModel
from diffusers import StableDiffusionTrainingPipeline
from diffusers.utils import export_to_gif
from diffusers import StableDiffusionTrainer, DreamBoothConfig
from diffusers import DreamBoothTrainer, DreamBoothConfig
from diffusers import StableDiffusionPipeline

from datasets import load_dataset

In [ ]:
# login to hugging face hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# load sample dataset from hugging face
dataset = load_dataset("lambdalabs/pokemon-blip-captions", split="train[:10]")
instance_prompt = "photo of sks dog"
class_prompt = "photo of a dog"

In [ ]:
# preprocess images into DreamBooth directory format
output_dir = "./dreambooth-dog-dataset"
instance_dir = os.path.join(output_dir, "instance_images")

os.makedirs(instance_dir, exist_ok=True)

In [ ]:
for i, item in enumerate(dataset):
    image = item["image"]
    image.save(os.path.join(instance_dir, f"{i}.png"))

In [ ]:
print("Dataset saved at:", instance_dir)

In [ ]:
# train DreamBooth model
output_model_dir = "./dreambooth-dog-output"

!accelerate config default

In [ ]:
# run DreamBooth training
config = DreamBoothConfig(
    pretrained_model_name_or_path="CompVis/stable-diffusion-v1-5",
    instance_data_dir=instance_dir,
    output_dir=output_model_dir,
    instance_prompt=instance_prompt,
    class_prompt=class_prompt,
    resolution=512,
    train_batch_size=1,
    max_train_steps=100,
    gradient_accumulation_steps=1,
    learning_rate=1e-6,
    lr_scheduler="constant",
    use_8bit_adam=True,
    gradient_checkpointing=True,
    prior_loss_weight=1.0,
)

In [ ]:
trainer = DreamBoothTrainer(config)
trainer.train()

In [ ]:
# load the model from local output directory
model_path = output_model_dir
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16).to("cuda")

In [ ]:
# define your custom prompt
prompt = "photo of sks dog wearing sunglasses, in a park"

In [ ]:
# generate an image
image = pipe(prompt).images[0]

In [ ]:
# display the image
plt.imshow(image)
plt.axis("off")
plt.title(prompt)
plt.show()